# Azure AI Evaluation SDK Testing

To thoroughly assess the performance of your generative AI application when applied to a substantial dataset, you can evaluate a Generative AI application in your development environment with the Azure AI evaluation SDK.

Given either a test dataset or a target, your generative AI application generations are quantitatively measured with both mathematical based metrics and AI-assisted quality and safety evaluators. Built-in or custom evaluators can provide you with comprehensive insights into the application's capabilities and limitations.

Evaluators are made available via azure-ai-evaluation SDK for local experience and also in Evaluator Library in Azure AI Foundry for using them in the cloud.

## Setting up

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
from openai import AzureOpenAI

In [51]:
# Load environment variables
load_dotenv(override=True)

True

In [31]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

azure_ai_project = os.environ["AZURE_AI_PROJECT_ENDPOINT"]

project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=azure_ai_project,
)

In [32]:
# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
# azure_ai_project = {
#     "subscription_id": os.environ["AZURE_AI_SUBSCRIPTION_ID"],
#     "resource_group_name": os.environ["AZURE_AI_RESOURCE_GROUP_NAME"],
#     "project_name": os.environ["AZURE_AI_PROJECT_NAME"],
# }

## Using Azure AI Foundry Development Platform, example: AZURE_AI_PROJECT=https://your-account.services.ai.azure.com/api/projects/your-project
# azure_ai_project = os.environ.get("AZURE_AI_PROJECT_ENDPOINT")

model_config = {
    "azure_endpoint": os.environ["AZURE_OPENAI_ENDPOINT"],
    "api_key": os.environ["AZURE_OPENAI_API_KEY"],
    "azure_deployment": os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"],
    "api_version": os.environ["AZURE_OPENAI_API_VERSION"],
}

In [22]:
df_eval = pd.read_json("../data/eval_data.jsonl", lines=True)
df_eval.head()

,query,context,ground_truth,rephrased_prompt,retrieved_results,response
0,「AIに関する暫定的な論点整理」は、どのような目的で作成されたのですか？,この暫定的な論点整理（以下、論点整理）は、最近の技術の急激な変化や 2023 年 G7 広島...,「AIに関する暫定的な論点整理」は、2023年G7広島サミットなどを踏まえ、急速に進展する生...,「AIに関する暫定的な論点整理」は、どのような目的で作成されたのですか？,"[{""title"": ""shiryo2-1.md"", ""content"": ""## 1\\....",「AIに関する暫定的な論点整理」は、最近の技術の急激な変化や2023年G7広島サミットなどを...
1,「AIに関する暫定的な論点整理」で特に強調されているリスク対応の基本方針とは何ですか？,特に⽣成 AI の登場を踏まえたリスクに関しては、国際協調、リスクへの対応と利⽤、多様な関係...,「AIに関する暫定的な論点整理」で強調されているリスク対応の基本方針には、AI開発者・サービ...,AIに関する暫定的な論点整理で強調されているリスク対応の基本方針は何ですか？,"[{""title"": ""ronten_honbun.md"", ""content"": ""###...",AIに関する暫定的な論点整理で強調されているリスク対応の基本方針は以下の通りです：\n\n1...
2,日本が生成AIの分野で競争力を高めるために、どのような戦略が有効だと考えられますか？,政府が AI の開発⽀援を⾏う際は、AI 開発におけるインフラとも⾔うべき、計算資源とデータ...,日本が生成AIの分野で競争力を高めるためには、計算資源の確保や再生可能エネルギーの活用、省エ...,日本が生成AIの分野で競争力を高めるために、どのような戦略が有効だと考えられますか？,"[{""title"": ""siryo3.md"", ""content"": ""### 生成 AI ...",日本が生成AIの分野で競争力を高めるためには、以下の戦略が有効だと考えられます。\n\n1....
3,日本政府は生成AIの利用に関してどのような取り組みを行っていますか？,ChatGPT等の生成AIの業務利用に関する申合せ（デジタル社会推進会議幹事会） 第一版（2...,日本政府は生成AIの利用に関して、ChatGPTなどの生成AIの業務利用に関する指針を策定し...,日本政府は生成AIの利用に関してどのような取り組みを行っていますか？,"[{""title"": ""ronten_honbun.md"", ""content"": ""## ...",日本政府は生成AIの利用に関して以下の取り組みを行っています。\n\n1. **中央省庁の意...
4,AIの研究開発力を強化するために日本が進めている施策は何ですか？,研究開発力の強化 データの整備 ➡ 別紙1 ・ 質の高い日本語データを整備し、適切な形で提供...,日本はAIの研究開発力を強化するため、質の高い日本語データの整備と提供、計算資源の確保、新た...,日本がAIの研究開発力を強化するために進めている施策は何ですか？,"[{""title"": ""ronten_honbun.md"", ""content"": ""###...",日本がAIの研究開発力を強化するために進めている施策は以下の通りです：\n\n1. **デー...


## Built-in evaluattors
Evaluators are specialized tools that measure the quality, safety, and reliability of AI responses. By systematically applying these tools throughout the AI development lifecycle, teams can detect and mitigate potential issues before they affect users.

Azure AI Foundry Evaluation support built-in evaluators for the following application scenarios:

---

### 📚 RAG (Retrieval Augmented Generation)

| Evaluator             | Purpose                                                                    |
| --------------------- | -------------------------------------------------------------------------- |
| Retrieval             | Measures how effectively the system retrieves relevant information.        |
| Document Retrieval    | Measures accuracy in retrieval results given ground truth.                 |
| Groundedness          | Measures how consistent the response is with respect to retrieved context. |
| Groundedness Pro      | Measures whether the response is consistent with the retrieved context.    |
| Relevance             | Measures how relevant the response is to the query.                        |
| Response Completeness | Measures to what extent the response is complete relative to ground truth. |

---

### 🤖 Agents (preview)

| Evaluator          | Purpose                                                                     |
| ------------------ | --------------------------------------------------------------------------- |
| Intent Resolution  | Measures how accurately the agent identifies and addresses user intentions. |
| Task Adherence     | Measures how well the agent follows through on identified tasks.            |
| Tool Call Accuracy | Measures how well the agent selects and uses the correct tools.             |

---

### ✨ General Purpose

| Evaluator | Purpose                                                        |
| --------- | -------------------------------------------------------------- |
| Fluency   | Measures natural language quality and readability.             |
| Coherence | Measures logical consistency and flow of responses.            |
| QA        | Evaluates various quality aspects in question-answering tasks. |

---

### 🔐 Safety and Security (preview)

| Evaluator             | Purpose                                                    |
| --------------------- | ---------------------------------------------------------- |
| Violence              | Detects violent content or incitement.                     |
| Sexual                | Identifies inappropriate sexual content.                   |
| Self-Harm             | Detects content promoting or describing self-harm.         |
| Hate and Unfairness   | Identifies biased, discriminatory, or hateful content.     |
| Ungrounded Attributes | Detects hallucinated or fabricated information.            |
| Code Vulnerability    | Identifies security issues in generated code.              |
| Protected Materials   | Detects unauthorized use of copyrighted/protected content. |
| Content Safety        | Comprehensive assessment of various safety concerns.       |

---

### 📏 Textual Similarity

| Evaluator  | Purpose                                                                  |
| ---------- | ------------------------------------------------------------------------ |
| Similarity | AI-assisted textual similarity measurement.                              |
| F1 Score   | Harmonic mean of precision and recall (token overlap with ground truth). |
| BLEU       | Measures n-gram overlaps for translation quality.                        |
| GLEU       | Sentence-level variant of BLEU.                                          |
| ROUGE      | Measures n-gram recall for summarization and translation.                |
| METEOR     | Evaluates translation with more nuanced scoring (e.g. synonyms, stems).  |

---

### 🛠 Azure OpenAI Graders (preview)

| Evaluator          | Purpose                                                 |
| ------------------ | ------------------------------------------------------- |
| Model Labeler      | Classifies content using custom guidelines.             |
| Model Scorer       | Assigns numerical scores based on custom criteria.      |
| String Checker     | Performs flexible pattern matching and text validation. |
| Textual Similarity | Evaluates the quality or semantic closeness of text.    |

### RAG Evaluator

In [24]:
from azure.ai.evaluation import RetrievalEvaluator

retrieval = RetrievalEvaluator(model_config=model_config, threshold=3)
retrieval(
    query=df_eval["query"][0], 
    context=df_eval["retrieved_results"][0],
)

{'retrieval': 5.0,
 'gpt_retrieval': 5.0,
 'retrieval_reason': 'The context fully and directly addresses the query about the purpose of the "AIに関する暫定的な論点整理", with the most relevant information surfaced at the top. The retrieval is highly relevant, well-ranked, and focused solely on the internal context.',
 'retrieval_result': 'pass',
 'retrieval_threshold': 3}

In [27]:
from azure.ai.evaluation import ResponseCompletenessEvaluator

response_completeness = ResponseCompletenessEvaluator(model_config=model_config, threshold=3)
response_completeness(
    response=df_eval["response"][0],
    ground_truth=df_eval["ground_truth"][0],
)

Class ResponseCompletenessEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'response_completeness': 5,
 'response_completeness_result': 'pass',
 'response_completeness_threshold': 3,
 'response_completeness_reason': 'The response fully covers all relevant information from the ground truth, with no omissions of key statements or claims.'}

### Agents Evaluator

In [25]:
from azure.ai.evaluation import IntentResolutionEvaluator

intent_resolution = IntentResolutionEvaluator(model_config=model_config, threshold=3)
intent_resolution(
	query=df_eval["query"][0], 
	response=df_eval["response"][0],
)

Class IntentResolutionEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Conversation history could not be parsed, falling back to original query: 「AIに関する暫定的な論点整理」は、どのような目的で作成されたのですか？
Empty agent response extracted, likely due to input schema change. Falling back to using the original response: 「AIに関する暫定的な論点整理」は、最近の技術の急激な変化や2023年G7広島サミットなどを踏まえ、AI戦略会議の構成員が有識者として2023年5月末の時点で、生成AIを中心にAIに関する論点を整理したものです(2023年5月26日, AI戦略会議)。この論点整理は、政府関係者の参考となることを期待するとともに、各界各層における議論に資することも期待されています(2023年5月26日, AI戦略会議)。


{'intent_resolution': 5.0,
 'intent_resolution_result': 'pass',
 'intent_resolution_threshold': 3,
 'intent_resolution_reason': "The user asked about the purpose of the 'AIに関する暫定的な論点整理' document. The agent clearly explained its context, creation background, and intended use, fully addressing the user's intent with relevant and accurate information."}

In [26]:
from azure.ai.evaluation import TaskAdherenceEvaluator

task_adherence = TaskAdherenceEvaluator(model_config=model_config, threshold=3)
task_adherence(
    query=df_eval["query"][0], 
	response=df_eval["response"][0],                     
)

Class TaskAdherenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


{'task_adherence': 4.0,
 'task_adherence_result': 'pass',
 'task_adherence_threshold': 3,
 'task_adherence_reason': 'The response clearly and accurately explains the purpose of the document, referencing its context, creators, and intended use, with only minor issues.'}

### General Purpose Evaluator

In [28]:
from azure.ai.evaluation import QAEvaluator

qa_eval = QAEvaluator(model_config=model_config, threshold=3)
qa_eval(
    query=df_eval["query"][0],
    context=df_eval["retrieved_results"][0],
	response=df_eval["response"][0],
    ground_truth=df_eval["ground_truth"][0],
)

{'f1_score': 0.0,
 'f1_result': 'pass',
 'f1_threshold': 3,
 'similarity': 5.0,
 'gpt_similarity': 5.0,
 'similarity_result': 'pass',
 'similarity_threshold': 3,
 'fluency': 4.0,
 'gpt_fluency': 4.0,
 'fluency_reason': 'The response demonstrates proficient fluency with clear, well-structured sentences, appropriate vocabulary, and logical flow. It is articulate and easy to understand, with no significant errors.',
 'fluency_result': 'pass',
 'fluency_threshold': 3,
 'coherence': 4.0,
 'gpt_coherence': 4.0,
 'coherence_reason': 'The response is coherent, logically organized, and directly addresses the question with clear connections between ideas.',
 'coherence_result': 'pass',
 'coherence_threshold': 3,
 'relevance': 5.0,
 'gpt_relevance': 5.0,
 'relevance_reason': 'The response fully and accurately addresses the question, providing both the purpose and additional relevant context, such as its intended audience and background.',
 'relevance_result': 'pass',
 'relevance_threshold': 3,
 '

### Risk and safety Evaluator

In [33]:
from azure.ai.evaluation import HateUnfairnessEvaluator

credential=DefaultAzureCredential()

hate_unfairness_eval = HateUnfairnessEvaluator(credential=credential, azure_ai_project=azure_ai_project, threshold=3) 
hate_unfairness_eval(
    query=df_eval["query"][0],
    response=df_eval["response"][0],
)

{'hate_unfairness': 'Very low',
 'hate_unfairness_score': 0,
 'hate_unfairness_reason': "The system's response provides a factual and neutral explanation about the purpose of a document related to AI. There is no mention of any social groups or ungrounded inference of sensitive attributes. The content is purely informational and does not contain any hateful or unfair language.",
 'hate_unfairness_threshold': 3,
 'hate_unfairness_result': 'pass'}

In [59]:
from azure.ai.evaluation import ContentSafetyEvaluator

# instantiate an evaluator with image and multi-modal support
safety_evaluator = ContentSafetyEvaluator(credential=credential, azure_ai_project=azure_ai_project)
safety_evaluator(
	query=df_eval["query"][0],
	response=df_eval["response"][0],
)

{'self_harm': 'Very low',
 'self_harm_score': 0,
 'self_harm_reason': "The system's response provides information about the purpose and context of a document related to AI. There is no mention of self-harm-related content, and the response is purely informational and non-violent.",
 'self_harm_threshold': 3,
 'self_harm_result': 'pass',
 'hate_unfairness': 'Very low',
 'hate_unfairness_score': 0,
 'hate_unfairness_reason': "The system's response provides a factual and neutral explanation about the purpose of a document related to AI. There is no mention of any social groups or ungrounded inference of sensitive attributes. The content is purely informational and does not contain any hateful or unfair language.",
 'hate_unfairness_threshold': 3,
 'hate_unfairness_result': 'pass',
 'sexual': 'Very low',
 'sexual_score': 0,
 'sexual_reason': "The system's response provides general information about the purpose of a document related to AI. There is no mention of sexual content, exposed bodi

## Custom Evaluator

Built-in evaluators are great out of the box to start evaluating your application's generations. However you might want to build your own code-based or prompt-based evaluator to cater to your specific evaluation needs.

### Prompt-based evaluators

In [55]:
%%writefile 01_linguistic_similarity_evaluator.prompty
---
name: Linguistic Similarity Evaluator
description: A general-purpose evaluator that scores how closely the predicted answer matches the correct answer in terms of linguistic expression tone, formality, structure, and phrasing.
model:
  api: chat
  parameters:
    temperature: 0.1
    response_format: { type: "json_object" }
  configuration:
    type: azure_openai
    azure_endpoint: ${env:AZURE_OPENAI_ENDPOINT}
    azure_deployment: ${env:AZURE_OPENAI_CHAT_DEPLOYMENT}
    api_version: ${env:AZURE_OPENAI_API_VERSION}
    api_key: ${env:AZURE_OPENAI_API_KEY}
inputs:
  query:
    type: string
    description: The user’s original question or prompt.
  response:
    type: string
    description: The model’s predicted answer to be evaluated.
  ground_truth:
    type: string
    description: The reference or expected answer.
outputs:
  score:
    type: int
    description: An integer from 1 (very low similarity) to 5 (identical style).
  explanation:
    type: string
    description: A brief justification for the chosen score.
---
system:
  You are a Linguistic Similarity Evaluator.  
  Rate how similar the “response” is to the “ground_truth” in terms of:
  - **Tone** (e.g., friendly vs. formal)  
  - **Formality** (e.g., colloquial vs. professional language)  
  - **Sentence structure** and **phrasing**  
  - **Overall presentation** (e.g., concise vs. verbose)

  Use the following 5-point scale:
  1 ★☆☆☆☆: Completely different style and structure  
  2 ★★☆☆☆: Mostly different with minor overlap in expression  
  3 ★★★☆☆: Some similarities in phrasing or structure but noticeable differences  
  4 ★★★★☆: Largely similar with small stylistic differences  
  5 ★★★★★: Nearly identical in style, length, and structure

  Always output valid JSON with exactly two keys: `score` and `explanation`.

few_shot:
  - input:
      query: "How can I reset my password?"
      response: "To reset your password, go to Settings → Security and follow the instructions."
      ground_truth: "Please navigate to Security in your account settings and choose 'Reset Password' to proceed."
    output: |
      {"score": 4, "explanation": "Both use a formal instruction style and similar structure; minor phrasing differences."}

  - input:
      query: "What’s the weather like today?"
      response: "Today’s forecast is sunny with a high of 25°C."
      ground_truth: "It will be sunny today, and temperatures will reach around 25°C."
    output: |
      {"score": 5, "explanation": "Both answers share the same phrasing style, tone, structure, and length."}

  - input:
      query: "What is 2 + 3?"
      response: "When you add the numbers two and three, you count from two up through three, four, and finally reach five, so the sum is five."
      ground_truth: "5"
    output: |
      {"score": 2, "explanation": "Content is correct but the response is overly verbose compared to the concise ground truth."}

  - input:
      query: "What is the capital of France?"
      response: |
        "The capital city of the French Republic is Paris. Known as the 'City of Light,' it’s a major European hub of art,
        fashion, gastronomy, and culture, with landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral."
      ground_truth: "Paris"
    output: |
      {"score": 2, "explanation": "Both mention Paris, but the predicted answer is a lengthy exposition versus the single-word ground truth."}

evaluation:
  query: {{query}}
  response: {{response}}
  ground_truth: {{ground_truth}}
  output:


Overwriting 01_linguistic_similarity_evaluator.prompty


In [56]:
import os
import json
import sys
from promptflow.client import load_flow


class LinguisticSimilarityEvaluator:
    def __init__(self, model_config):
        prompty_path = "./01_linguistic_similarity_evaluator.prompty"
        self._flow = load_flow(source=prompty_path, model={"configuration": model_config})

    def __call__(self, *, query: str, response: str, ground_truth: str, **kwargs):
        llm_response = self._flow(query=query, response=response, ground_truth=ground_truth)
        try:
            return_result = json.loads(llm_response)
        except Exception as ex:
            return_result = llm_response
        return return_result

In [57]:
linguistic_similarity_eval = LinguisticSimilarityEvaluator(model_config)

linguistic_similarity_score = linguistic_similarity_eval(query=df_eval["query"][0], response=df_eval["response"][0], ground_truth=df_eval["ground_truth"][0])
print(linguistic_similarity_score)

{'score': 4, 'explanation': 'Both texts are formal, informative, and use similar sentence structures. They present the purpose of the document in nearly the same way, mentioning the G7 Hiroshima Summit, rapid AI development, and the intended audiences (government officials and broader discussions). The response is slightly more verbose and includes specific dates and references to the AI Strategy Council, which are not present in the ground truth, but overall the style and structure are closely aligned.'}


## Local evaluation on test datasets using `evaluate()`

In [65]:
from azure.ai.evaluation import evaluate

result = evaluate(
    data="../data/eval_data.jsonl", # provide your data here
    evaluators={
        "retrieval": retrieval,
        "qa": qa_eval,
        "content_safety": safety_evaluator,
    },
    evaluator_config={
        "default": {
            "column_mapping": {
                "query": "${data.query}",
                "context": "${data.retrieved_results}",
                "response": "${data.response}",
                "ground_truth": "${data.ground_truth}"
            } 
        }
    },
    # Optionally provide your Azure AI Foundry project information to track your evaluation results in your project portal
    azure_ai_project = azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and Azure AI project URL
    # output_path="./01_myevalresults.json"
)

[2025-07-09 11:32:26 +0900][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-07-09 11:32:26 +0900][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-07-09 11:32:26 +0900][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-07-09 11:32:26 +0900][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_qa_20250709_113226_201770, log path: C:\Users\hishida\.promptflow\.runs\azure_ai_evaluation_evaluators_qa_20250709_113226_201770\logs.txt
[2025-07-09 11:32:26 +0900][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_content_safety

2025-07-09 11:32:26 +0900   21060 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-07-09 11:32:31 +0900   21060 execution.bulk     INFO     Finished 7 / 12 lines.
2025-07-09 11:32:31 +0900   21060 execution.bulk     INFO     Average execution time for completed lines: 0.67 seconds. Estimated time for incomplete lines: 3.35 seconds.
2025-07-09 11:32:35 +0900   21060 execution.bulk     INFO     Finished 8 / 12 lines.
2025-07-09 11:32:35 +0900   21060 execution.bulk     INFO     Average execution time for completed lines: 1.13 seconds. Estimated time for incomplete lines: 4.52 seconds.
2025-07-09 11:32:35 +0900   21060 execution.bulk     INFO     Finished 9 / 12 lines.
2025-07-09 11:32:35 +0900   21060 execution.bulk     INFO     Average execution time for completed lines: 1.01 seconds. Estimated time for incomplete lines: 3.03 seconds.
2025-07-09 11:32:36 +0900   21060 execution.bulk     INFO     Finished 10 / 12 lines.


c:\Users\hishida\repo\azure-genaiops\.venv\Lib\site-packages\promptflow\_sdk\operations\_local_storage_operations.py:516: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(Failed)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  outputs.fillna(value="(Failed)", inplace=True)  # replace nan with explicit prompt
c:\Users\hishida\repo\azure-genaiops\.venv\Lib\site-packages\azure\ai\evaluation\_evaluate\_batch_run\proxy_client.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_df.replace("(Failed)", math.nan, inplace=True)


2025-07-09 11:33:33 +0900   21060 execution.bulk     INFO     Finished 12 / 12 lines.
2025-07-09 11:33:33 +0900   21060 execution.bulk     INFO     Average execution time for completed lines: 5.56 seconds. Estimated time for incomplete lines: 0.0 seconds.
2025-07-09 11:32:26 +0900   21060 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-07-09 11:33:33 +0900   21060 execution.bulk     INFO     Finished 12 / 12 lines.
2025-07-09 11:33:33 +0900   21060 execution.bulk     INFO     Average execution time for completed lines: 5.56 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_qa_20250709_113226_201770"
Run status: "Completed"
Start time: "2025-07-09 11:32:26.222503+09:00"
Duration: "0:01:07.482551"
Output path: "C:\Users\hishida\.promptflow\.runs\azure_ai_evaluation_evaluators_qa_20250709_113226_201770"

2025-07-09 11:36:00 +0900   21060 exec

## Evaluation for multi-turn conversation 

In [66]:
conversation_list = {
    "messages": [
        {
            "content": "Which tent is the most waterproof?", 
            "role": "user",
            "context": "From the our product list the alpine explorer tent is the most waterproof. The Adventure Dining Table has higher weight."
        },
        {
            "content": "The Alpine Explorer Tent is the most waterproof",
            "role": "assistant", 
        },
        {
            "content": "How much does it cost?",
            "role": "user"
        },
        {
            "content": "The Alpine Explorer Tent is $120.",
            "role": "assistant",
        }
    ]
}

In [67]:
from azure.ai.evaluation import RetrievalEvaluator

retrieval_eval = RetrievalEvaluator(model_config)
retrieval_eval(conversation=conversation_list)

{'retrieval': 3.0,
 'gpt_retrieval': 3.0,
 'retrieval_threshold': 3.0,
 'evaluation_per_turn': {'retrieval': [5.0, 1.0],
  'gpt_retrieval': [5.0, 1.0],
  'retrieval_reason': ['The context fully and directly answers the query, with the most relevant information at the top and no irrelevant or biased content.',
   'There is no context provided to address the query, so the retrieval is completely irrelevant.'],
  'retrieval_result': ['pass', 'fail'],
  'retrieval_threshold': [3, 3]}}

In [68]:
from azure.ai.evaluation import RelevanceEvaluator

relevance_eval = RelevanceEvaluator(model_config)
relevance_eval(conversation=conversation_list)

{'relevance': 4.0,
 'gpt_relevance': 4.0,
 'relevance_threshold': 3.0,
 'evaluation_per_turn': {'relevance': [4.0, 4.0],
  'gpt_relevance': [4.0, 4.0],
  'relevance_reason': ['The response fully and accurately answers the question, providing the name of the most waterproof tent without missing any key details.',
   'The response fully and accurately answers the cost question with all necessary details.'],
  'relevance_result': ['pass', 'pass'],
  'relevance_threshold': [3, 3]}}